## Scraping CoinMarketCap to produce a crypto portfolio valuation

### Section 1. Overview and project scope

[Coinmarketcap](https://coinmarketcap.com/) is a website that provides real-time data on [cryptocurrencies](https://en.wikipedia.org/wiki/Cryptocurrency). The scope of this project is to leverage Python to:

1. retrieve real-time crypto currency prices from [Coinmarketcap](https://coinmarketcap.com)
2. retrieve real-time foreign exchange rates from [Google Finance](https://www.google.com/finance/)
3. add struture to the retrieved data by creating custom Classes  
4. allow the user to select a portfolio of cryptocurrencies and holdings
5. provide a real-time portfolio valuation based on the user selelction

### Section 2. Import the required libraries

In [1]:
import requests as r
import re
from bs4 import BeautifulSoup
from dataclasses import dataclass
from typing import List
import datetime as dt

In [2]:
now = dt.datetime.now()
print(now)

2025-01-17 14:42:26.131420
